## This Notebook is our PlayGround

### Module Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from dotenv import load_dotenv

import base64
import requests
import json

from github import Github,Auth
import google.generativeai as genai

In [4]:
load_dotenv()

True

In [5]:
def create_and_get_gemini_model():
    genai.configure(api_key=os.getenv('GOOGLE_GEMINI_API_KEY'))
    model = genai.GenerativeModel('gemini-pro')
    return model

In [15]:
def create_get_github_object():
    g=Github(auth=Auth.Token(os.getenv('GITHUB_ACCESS_TOKEN')))
    return g

In [16]:
def get_username_and_repo_from_url(url):
    starting='https://github.com/'
    return url[len(starting):]

In [19]:
def get_repository_object(github_object,repo_url):
    uname_and_reponame = get_username_and_repo_from_url(repo_url)
    repo_object = github_object.get_repo(uname_and_reponame)
    return repo_object

In [11]:
def load_prompt(file_path='./prompt.txt'):
    with open(file_path,'r') as f:
        prompt=f.read()
    return prompt

In [32]:
def get_report_from_gemini(model,code_file):
    prompt=load_prompt()
    report = model.generate_content(f"""{prompt}/n{code_file}""")
    return report.text

In [13]:
def decode_bytes_to_str(byte_data):
    decoded = base64.b64decode(byte_data)
    return decoded

In [17]:
dirs_to_ignore=['node_modules','.github']
files_to_ignore=['LICENSE','.gitignore','.git','.gitattributes','.env','.bat','.sh']
extensions_to_ignore=['md','txt','pptx','docx','xlsx','csv','db','sqlite','yaml','abi','bin','json','onnx','jpg','png','jpeg']

In [21]:
def get_list_of_all_files_in_repo(repo_object):
    contents= repo_object.get_contents("")
    files=[]
    while contents:
        item = contents.pop(0)
        if (item.type == "dir" and item.name not in dirs_to_ignore):
            contents.extend(repo_object.get_contents(item.path))
        elif (item.type=="file" and item.name not in files_to_ignore and item.name.split(".")[-1] not in extensions_to_ignore):
            files.append(item.path)
    return files

In [22]:
def get_file_content_from_repo(repo_object,file_name):
    encoded_file_content = repo_object.get_contents(file_name.content).content
    decoded_file_content = decode_bytes_to_str(encoded_file_content)
    return decoded_file_content